<a href="https://colab.research.google.com/github/creativesar/Guess-the-number-Computer-/blob/main/Guess_the_number_by_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import os

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def display_header():
    header = """
    ╔═══════════════════════════════════════╗
    ║       GUESS THE NUMBER CHALLENGE       ║
    ╚═══════════════════════════════════════╝
    """
    print(header)

def get_difficulty():
    display_header()
    print("Choose your difficulty level:")
    print("1. Easy   (1-50,  15 attempts)")
    print("2. Medium (1-100, 10 attempts)")
    print("3. Hard   (1-200, 8 attempts)")
    print("4. Expert (1-500, 6 attempts)")

    while True:
        try:
            choice = int(input("Enter your choice (1-4): "))
            if 1 <= choice <= 4:
                if choice == 1:
                    return 1, 50, 15
                elif choice == 2:
                    return 1, 100, 10
                elif choice == 3:
                    return 1, 200, 8
                else:
                    return 1, 500, 6
            else:
                print("Please enter a number between 1 and 4.")
        except ValueError:
            print("Please enter a valid number.")

def calculate_score(attempts, max_attempts, range_size):
    # Base score depends on difficulty
    base_score = range_size // 10
    # More points for using fewer attempts
    attempt_bonus = int((max_attempts - attempts + 1) / max_attempts * 100)
    return base_score * attempt_bonus

def get_hint(secret_number, previous_guesses, hint_level):
    if len(previous_guesses) < 1:
        return "Make at least one guess first!"

    hints = [
        # Level 1 hints (basic)
        [
            f"The number is {'even' if secret_number % 2 == 0 else 'odd'}.",
            f"The number is {'' if secret_number > 50 else 'not '}greater than 50.",
            f"The last digit of the number is {secret_number % 10}."
        ],
        # Level 2 hints (medium)
        [
            f"The sum of the digits is {sum(int(digit) for digit in str(secret_number))}.",
            f"The number {'is' if secret_number in [i**2 for i in range(1, 25)] else 'is not'} a perfect square.",
            f"The number {'is' if secret_number % 10 == 0 else 'is not'} a multiple of 10."
        ],
        # Level 3 hints (advanced)
        [
            f"The number has {len(str(secret_number))} digits.",
            f"The first digit is {str(secret_number)[0]}.",
            f"The number {'is' if secret_number in [i for i in range(1, 101) if all(i % j != 0 for j in range(2, int(i**0.5) + 1))] else 'is not'} a prime number."
        ]
    ]

    # Choose a random hint from the appropriate level
    if hint_level <= len(hints):
        return random.choice(hints[hint_level-1])
    else:
        # If hint level is too high, give a very specific hint
        return f"The number is between {secret_number-5} and {secret_number+5}."

def get_temperature(guess, secret_number, max_range):
    """Returns how 'hot' or 'cold' the guess is"""
    difference = abs(secret_number - guess)
    percentage = (difference / max_range) * 100

    if percentage < 2:
        return "🔥 BURNING HOT! 🔥"
    elif percentage < 5:
        return "🔥 Very hot! 🔥"
    elif percentage < 10:
        return "🔥 Hot!"
    elif percentage < 20:
        return "🌡️ Warm"
    elif percentage < 30:
        return "❄️ Cool"
    elif percentage < 50:
        return "❄️ Cold"
    else:
        return "❄️ FREEZING! ❄️"

def guess_the_number():
    high_score = 0
    total_score = 0
    games_played = 0

    while True:
        clear_screen()
        min_num, max_num, max_attempts = get_difficulty()
        secret_number = random.randint(min_num, max_num)
        attempts = 0
        previous_guesses = []
        hints_used = 0
        hint_level = 1

        clear_screen()
        display_header()
        print(f"I'm thinking of a number between {min_num} and {max_num}.")
        print(f"You have {max_attempts} attempts to guess it.")
        print(f"Current high score: {high_score}")

        while attempts < max_attempts:
            try:
                print("\nOptions:")
                print("- Enter a number to make a guess")
                print(f"- Type 'hint' for a level {hint_level} clue (costs an attempt)")
                print("- Type 'history' to see your previous guesses")
                print("- Type 'help' for game instructions")

                user_input = input("\nYour choice: ").lower()

                if user_input == 'hint':
                    attempts += 1
                    hints_used += 1
                    hint = get_hint(secret_number, previous_guesses, hint_level)
                    print(f"HINT: {hint}")
                    print(f"That cost you an attempt. {max_attempts - attempts} attempts remaining.")

                    # Increase hint level for next time
                    if hint_level < 3:
                        hint_level += 1
                    continue

                elif user_input == 'history':
                    if previous_guesses:
                        print(f"Your previous guesses: {', '.join(map(str, previous_guesses))}")

                        # Show temperature for last guess
                        if len(previous_guesses) > 0:
                            last_guess = previous_guesses[-1]
                            temp = get_temperature(last_guess, secret_number, max_num - min_num)
                            print(f"Your last guess ({last_guess}) was: {temp}")
                    else:
                        print("You haven't made any guesses yet.")
                    continue

                elif user_input == 'help':
                    print("\n=== GAME INSTRUCTIONS ===")
                    print("1. Guess the secret number within the given attempts")
                    print("2. After each guess, you'll be told if your guess is too high or too low")
                    print("3. You can use hints, but each hint costs one attempt")
                    print("4. Higher difficulty levels give more points")
                    print("5. Using fewer attempts gives you a higher score")
                    print("6. The 'history' command shows your previous guesses")
                    print("7. Good luck!")
                    continue

                guess = int(user_input)

                if guess < min_num or guess > max_num:
                    print(f"Please enter a number between {min_num} and {max_num}.")
                    continue

                if guess in previous_guesses:
                    print("You already guessed that number! Try a different one.")
                    continue

                previous_guesses.append(guess)
                attempts += 1

                # Dramatic pause for effect
                print("Checking your guess", end="")
                for _ in range(3):
                    time.sleep(0.3)
                    print(".", end="", flush=True)
                print()

                # Check the guess
                difference = abs(secret_number - guess)
                if guess < secret_number:
                    if difference > (max_num - min_num) // 4:
                        print("Way too low! ↓↓↓")
                    else:
                        print("Too low! ↓")

                    # Add temperature indicator
                    temp = get_temperature(guess, secret_number, max_num - min_num)
                    print(temp)

                elif guess > secret_number:
                    if difference > (max_num - min_num) // 4:
                        print("Way too high! ↑↑↑")
                    else:
                        print("Too high! ↑")

                    # Add temperature indicator
                    temp = get_temperature(guess, secret_number, max_num - min_num)
                    print(temp)

                else:
                    print("\n🎉 CONGRATULATIONS! 🎉")
                    print(f"You guessed the number in {attempts} attempts!")

                    # Calculate and update score
                    current_score = calculate_score(attempts, max_attempts, max_num - min_num)
                    if hints_used > 0:
                        hint_penalty = hints_used * 20
                        print(f"Hint penalty: -{hint_penalty} points")
                        current_score = max(0, current_score - hint_penalty)

                    print(f"You earned {current_score} points!")
                    total_score += current_score

                    if current_score > high_score:
                        print(f"NEW HIGH SCORE! Previous: {high_score}, New: {current_score}")
                        high_score = current_score

                    break

                # Inform player of remaining attempts
                remaining = max_attempts - attempts
                if remaining > 0:
                    print(f"You have {remaining} attempts left.")

                    # Give encouragement when running low on attempts
                    if remaining <= 2:
                        print("You're running out of attempts! Focus!")
                        # Offer a free hint when down to last attempts
                        if remaining == 1 and hints_used == 0:
                            print("Here's a free hint to help you:")
                            print(f"HINT: {get_hint(secret_number, previous_guesses, 1)}")

            except ValueError:
                print("Please enter a valid number or command.")

        if attempts >= max_attempts and guess != secret_number:
            print(f"\n😢 GAME OVER! The number was {secret_number}.")

        games_played += 1
        print(f"\nGames played: {games_played}")
        print(f"Total score: {total_score}")
        print(f"Average score: {total_score / games_played:.1f}")

        # Ask if player wants to play again
        play_again = input("\nDo you want to play again? (yes/no): ").lower()
        if play_again not in ["yes", "y"]:
            print("\nThanks for playing! Final stats:")
            print(f"Games played: {games_played}")
            print(f"High score: {high_score}")
            print(f"Total score: {total_score}")
            print(f"Average score: {total_score / games_played:.1f}")
            print("\nGoodbye! 👋")
            break

if __name__ == "__main__":
    guess_the_number()


    ╔═══════════════════════════════════════╗
    ║       GUESS THE NUMBER CHALLENGE       ║
    ╚═══════════════════════════════════════╝
    
Choose your difficulty level:
1. Easy   (1-50,  15 attempts)
2. Medium (1-100, 10 attempts)
3. Hard   (1-200, 8 attempts)
4. Expert (1-500, 6 attempts)
Enter your choice (1-4): 1

    ╔═══════════════════════════════════════╗
    ║       GUESS THE NUMBER CHALLENGE       ║
    ╚═══════════════════════════════════════╝
    
I'm thinking of a number between 1 and 50.
You have 15 attempts to guess it.
Current high score: 0

Options:
- Enter a number to make a guess
- Type 'hint' for a level 1 clue (costs an attempt)
- Type 'history' to see your previous guesses
- Type 'help' for game instructions

Your choice: hint
HINT: Make at least one guess first!
That cost you an attempt. 14 attempts remaining.

Options:
- Enter a number to make a guess
- Type 'hint' for a level 2 clue (costs an attempt)
- Type 'history' to see your previous guesses
- Type 'h